In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# Загрузка данных
data = pd.read_csv(r'C:\Users\Oleg\Documents\bank\bank-full.csv', delimiter=";")

# Выбор указанных признаков
data_selected = data[['age', 'job', 'marital', 'education', 'balance', 'housing', 'contact',
                      'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']]

# Проверка пропущенных значений
print("Пропущенные значения:\n", data_selected.isnull().sum())

# Вопрос 1: Самое частое значение (mode) для столбца education
education_mode = data_selected['education'].mode()[0]
print("Самое частое значение для education:", education_mode)

# Вопрос 2: Корреляционная матрица для числовых признаков
numeric_features = data_selected[['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']]
correlation_matrix = numeric_features.corr()
print("Корреляционная матрица:\n", correlation_matrix)

# Поиск пары признаков с наибольшей корреляцией
correlation_pairs = correlation_matrix.unstack()
sorted_correlation = correlation_pairs.sort_values(ascending=False)
strongest_correlation = sorted_correlation[(sorted_correlation != 1.0)].idxmax()
print("Наибольшая корреляция между признаками:", strongest_correlation)

# Кодирование целевой переменной y
data_selected.loc[:, 'y'] = data_selected['y'].map({'yes': 1, 'no': 0})

print("Уникальные значения в y после кодирования:", data_selected['y'].unique())

# Разделение данных на обучающую, валидационную и тестовую выборки
train_data, temp_data = train_test_split(data_selected, test_size=0.4, random_state=42, stratify=data_selected['y'])
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42, stratify=temp_data['y'])

# Разделение признаков и целевой переменной
X_train = train_data.drop(columns=['y'])
y_train = train_data['y']
X_val = val_data.drop(columns=['y'])
y_val = val_data['y']
X_test = test_data.drop(columns=['y'])
y_test = test_data['y']

# Преобразуем целевые переменные в целочисленный тип данных
y_train = y_train.astype(int)
y_val = y_val.astype(int)
y_test = y_test.astype(int)

# Вопрос 3: Взаимная информация между y и категориальными переменными
categorical_features = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']
X_train_encoded = pd.get_dummies(X_train, columns=categorical_features)
mi_scores = mutual_info_classif(X_train_encoded, y_train, discrete_features=True)
mi_results = pd.Series(mi_scores, index=X_train_encoded.columns).round(2)

# Отображаем результаты для всех признаков с наибольшей взаимной информацией
print("Топ взаимной информации для категориальных признаков:\n", mi_results.sort_values(ascending=False).head(10))

# Вопрос 4: Логистическая регрессия и точность на валидационном наборе
X_val_encoded = pd.get_dummies(X_val, columns=categorical_features)
X_train_encoded, X_val_encoded = X_train_encoded.align(X_val_encoded, join='left', axis=1, fill_value=0)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_encoded, y_train)
y_val_pred = model.predict(X_val_encoded)

# Рассчитываем и выводим точность
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Точность на валидационном наборе данных:", round(val_accuracy, 2))
# Шаг 1: Обучаем модель на всех признаках и вычисляем исходную точность
model_full = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model_full.fit(X_train_encoded, y_train)
y_val_pred_full = model_full.predict(X_val_encoded)
accuracy_full = accuracy_score(y_val, y_val_pred_full)
print("Исходная точность модели на всех признаках:", round(accuracy_full, 2))

# Словарь для хранения разницы точности
accuracy_diff = {}

# Шаг 2: Последовательно исключаем каждый признак и измеряем изменение точности
features_to_check = ['age', 'balance', 'marital', 'previous']  # Признаки, указанные в вопросе

for feature in features_to_check:
    # Исключаем признак из тренировочного и валидационного наборов
    X_train_reduced = X_train_encoded.drop(columns=[feature], errors='ignore')
    X_val_reduced = X_val_encoded.drop(columns=[feature], errors='ignore')

    # Обучаем модель без текущего признака
    model_reduced = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model_reduced.fit(X_train_reduced, y_train)

    # Рассчитываем точность модели без текущего признака
    y_val_pred_reduced = model_reduced.predict(X_val_reduced)
    accuracy_reduced = accuracy_score(y_val, y_val_pred_reduced)

    # Вычисляем разницу в точности и сохраняем результат
    accuracy_diff[feature] = accuracy_full - accuracy_reduced
    print(f"Разница в точности при исключении {feature}: {round(accuracy_diff[feature], 4)}")

# Шаг 3: Находим признак с наименьшей разницей в точности
least_impact_feature = min(accuracy_diff, key=accuracy_diff.get)
print(f"\nПризнак с наименьшей разницей в точности: {least_impact_feature}")
# Массив значений параметра C
C_values = [0.01, 0.1, 1, 10, 100]

# Словарь для хранения точностей для каждого значения C
accuracy_by_C = {}

# Шаг 1: Обучаем модели с разными значениями C и вычисляем точность на валидационном наборе
for C in C_values:
    # Инициализация модели логистической регрессии с заданным значением C
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)

    # Обучение модели
    model.fit(X_train_encoded, y_train)

    # Предсказания на валидационном наборе
    y_val_pred = model.predict(X_val_encoded)

    # Рассчитываем точность и сохраняем результат в словарь
    accuracy = accuracy_score(y_val, y_val_pred)
    accuracy_by_C[C] = round(accuracy, 3)
    print(f"Точность для C={C}: {accuracy_by_C[C]}")

# Шаг 2: Находим значение C с наивысшей точностью (и выбираем наименьшее, если есть несколько максимальных)
best_C = min([k for k, v in accuracy_by_C.items() if v == max(accuracy_by_C.values())])
print(f"\nЗначение C с наилучшей точностью: {best_C}")